In [1]:

import rospy
import time
import jupyros as jr
import matplotlib
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import math
import actionlib
import actionlib.msg

%matplotlib widget

from matplotlib.animation import FuncAnimation
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from std_msgs.msg import String, Float64, Bool, Float32
from rt2_assignment1.srv import Command
from ipywidgets import interact,interactive,fixed,interact_manual
from geometry_msgs.msg import Twist

In [2]:
rospy.init_node('user_interface')
client=rospy.ServiceProxy('/user_interface', Command)
Vel_Publisher = rospy.Publisher('/vel', Twist, queue_size = 1)
Command_Publisher = rospy.Publisher('/cmd_vel', Twist, queue_size = 1)
velocity = Twist()
speed = Twist()
starting_time = time.time()
finished = interrupted = 0
t = []
t1 = []
x = []
y = []
vel_lin_cmd = []
vel_ang_cmd = []
vel_lin_speed = []
vel_ang_speed = []
stats= [0,0]
Vel_linear = 0
Vel_angular = 0

In [3]:
def Random_start(r):
    global random_behaviour
    random_behaviour = True
    client('start')

def Random_stop(r):
    global random_behaviour
    random_behaviour = False
    client('stop')

def Onwards(t):
    global random_behaviour, speed, velocity
    if random_behaviour:
        random_behaviour = False
        client('stop')
    Stop(1)
    velocity.linear.x = speed.linear.x

    Command_Publisher.publish(velocity)
    
def Backwards(t):
    global random_behaviour, speed, velocity
    if random_behaviour:
        random_behaviour = False
        client('stop')
    Stop(1)
    velocity.linear.x = - speed.linear.x

    Command_Publisher.publish(velocity)

def leftwards(t):
    global random_behaviour, speed, velocity
    if random_behaviour:
        random_behaviour = False
        client('stop')
    Stop(1)
    velocity.angular.z = - speed.angular.z

    Command_Publisher.publish(velocity)

def rightwards(t):
    global random_behaviour, speed, velocity
    if random_behaviour:
        random_behaviour = False
        client('stop')
    Stop(1)
    velocity.angular.z = speed.angular.z

    Command_Publisher.publish(velocity)

def Stop(s):
    velocity.linear.x = 0
    velocity.angular.z = 0
    Command_Publisher.publish(velocity)

def Set_Linear_Velocity(vel):
    global speed
    speed.linear.x = vel
    Vel_Publisher.publish(speed)
    return vel

def Set_Angular_Velocity(vel):
    global speed
    speed.angular.z = vel
    Vel_Publisher.publish(speed)
    return vel

In [4]:
Button_rand_go=widgets.Button(
    description='Start Random Behaviour',
    style=widgets.ButtonStyle(button_color='blue'),
    layout=widgets.Layout(width='auto', grid_area='but_rand_go')
)

Button_rand_stop=widgets.Button(
    description='Stop Random Behaviour',
    style=widgets.ButtonStyle(button_color='purple'),
    layout=widgets.Layout(width='auto', grid_area='but_rand_stop')
)

Button_onwards=widgets.Button(
    description='Onwards',
    style=widgets.ButtonStyle(button_color='pink'),
    layout=widgets.Layout(width='auto', grid_area='but_onwards')
)

Button_backwards=widgets.Button(
    description='Backwards',
    style=widgets.ButtonStyle(button_color='pink'),
    layout=widgets.Layout(width='auto', grid_area='but_backwards')
)

Button_Leftwards=widgets.Button(
    description='Left',
    style=widgets.ButtonStyle(button_color='pink'),
    layout=widgets.Layout(width='auto', grid_area='but_left')
)

Button_Rightwards=widgets.Button(
    description='Right',
    style=widgets.ButtonStyle(button_color='pink'),
    layout=widgets.Layout(width='auto', grid_area='but_right')
)

Button_stop=widgets.Button(
    description='Stop',
    style=widgets.ButtonStyle(button_color='red'),
    layout=widgets.Layout(width='auto', grid_area='but_stop')
)


In [5]:
Button_rand_go.on_click(Random_start)
Button_rand_stop.on_click(Random_stop)
Button_onwards.on_click(Onwards)
Button_backwards.on_click(Backwards)
Button_Leftwards.on_click(leftwards)
Button_Rightwards.on_click(rightwards)
Button_stop.on_click(Stop)

In [6]:
print("Linear Velocity Value")
interact(Set_Linear_Velocity, vel =widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5))

print("Angular Velocity Value")
interact(Set_Angular_Velocity, vel =widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5))

widgets.GridBox(children=[Button_rand_go, Button_rand_stop, Button_onwards, Button_backwards, Button_Leftwards, Button_Rightwards, Button_stop],
                layout=widgets.Layout(width='80%',grid_template_rows='auto auto',
                grid_template_columns='32% 16% 16% 16% 16% ',
                grid_template_areas='''"but_rand_go . . but_onwards but_stop ""but_rand_stop . but_left but_backwards but_right "''')
               )

Linear Velocity Value


interactive(children=(FloatSlider(value=0.5, description='vel', max=1.0, step=0.01), Output()), _dom_classes=(…

Angular Velocity Value


interactive(children=(FloatSlider(value=0.5, description='vel', max=1.0, step=0.01), Output()), _dom_classes=(…

GridBox(children=(Button(description='Start Random Behaviour', layout=Layout(grid_area='but_rand_go', width='a…

In [7]:
from tf import transformations



def normalize_angle(angle):
    if(math.fabs(angle) > math.pi):
        angle = angle - (2 * math.pi * angle) / (math.fabs(angle))
    return angle

def clbk_odom(msg):
    global position_
    global yaw_
    global t1, time_elapsed, x, y, vel_lin_cmd, vel_lin_speed, vel_ang_cmd, vel_ang_speed, dir_x, dir_y
    time_elapsed = time.time() - starting_time

    if len(t1)>500:
        t1.pop(0)
        x.pop(0)
        y.pop(0)
        vel_lin_cmd.pop(0)
        vel_lin_speed.pop(0)
        vel_ang_cmd.pop(0)
        vel_ang_speed.pop(0)
    # position
    position_ = msg.pose.pose.position
    vel_ = msg.twist.twist

    t1.append(time_elapsed)
    x.append(position_.x)
    y.append(position_.y)
    vel_lin_cmd.append(Vel_linear)
    vel_ang_cmd.append(Vel_angular)
    vel_lin_speed.append(vel_.linear.x)
    vel_ang_speed.append(vel_.angular.z)
    
    # yaw
    quaternion = (
        msg.pose.pose.orientation.x,
        msg.pose.pose.orientation.y,
        msg.pose.pose.orientation.z,
        msg.pose.pose.orientation.w)
    euler = transformations.euler_from_quaternion(quaternion)
    yaw_ = euler[2]

    yaw = normalize_angle(yaw_)
    dir_x = math.cos(yaw)
    dir_y = math.sin(yaw)

def target_clbk(msg):
    global finished, interrupted, t, stats
    if msg.data == -1:
        interrupted += 1
        print("1")
    else: 
        finished += 1
        print("2")
        t.append(msg.data)
    print("target")
    stats = [finished, interrupted]

def Velocity_clbk(msg):
    global Vel_linear, Vel_angular
    Vel_linear = msg.linear.x
    Vel_angular = msg.angular.z


Command_Subscriber = rospy.Subscriber('/cmd_vel', Twist, Velocity_clbk)
odom_Sub = rospy.Subscriber('/odom', Odometry, clbk_odom)
target_Sub = rospy.Subscriber('/target', Float32, target_clbk)



In [8]:
figure1,axis1=plt.subplots()
axis1.set_ylim((-3,3))
#axis1.set_xlim((0,15))
linea=axis1.plot([],[],lw=1,color='red',label='Actual Velocity')[0]
lineb=axis1.plot([],[],lw=1,color='blue',label='Commanded Velocity')[0]
line1,=axis1.plot([],[], lw=1)
axis1.legend()
axis1.set_title('Linear Velocity')
axis1.set_xlabel('Time')
axis1.set_ylabel('Meters/Seconds')
lines1_vec=[]
lines1_vec.append(linea)
lines1_vec.append(lineb)

figure2,axis2=plt.subplots()
axis2.set_ylim((-3,3))
#axis2.set_xlim((0,15))
linec=axis2.plot([],[],lw=1,color='red',label='Actual Velocity')[0]
lined=axis2.plot([],[],lw=1,color='blue',label='Commanded Velocity')[0]
line2,=axis2.plot([],[], lw=1)
axis2.legend()
axis2.set_title('Angular Velocity')
axis2.set_xlabel('Time')
axis2.set_ylabel('Radiants/Seconds')
lines2_vec=[]
lines2_vec.append(linec)
lines2_vec.append(lined)

def initialization1():
    line1.set_data([],[])

def initialization2():
    line2.set_data([],[])

def anim1(b):
    axis1.set_xlim((time_elapsed-5, time_elapsed+5))
    vec_x = [t1, t1]
    vec_y = [vel_lin_speed, vel_lin_cmd]

    for i,line1 in enumerate(lines1_vec):
        line1.set_data(vec_x[i], vec_y[i])
    return (lines1_vec)

def anim2(a):
    axis2.set_xlim((time_elapsed-5, time_elapsed+5))
    vec_x = [t1, t1]
    vec_y = [vel_ang_speed, vel_ang_cmd]

    for j,line2 in enumerate(lines2_vec):
        line2.set_data(vec_x[j], vec_y[j])
    return (lines2_vec)

animation1=animation.FuncAnimation(figure1, anim1, init_func = initialization1, frames=100, interval=100, blit=True)
animation2=animation.FuncAnimation(figure2, anim2, init_func = initialization2, frames=100, interval=100, blit=True)





Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
figure3,axis3 =  plt.subplots()
legend = ['Finished', 'Interrupted']
#axis3.bar(legend,stats,color = 'r', width = 0.5)
axis3.set_title("Finished and Interrupted goal")

def initialization3():
    axis3.bar(legend,stats,color = 'r', width = 0.5)

def anim3(i):
    global axis3
    axis3.bar(legend,stats,color = 'r', width = 0.5)

animation3=animation.FuncAnimation(figure3, anim3, init_func=initialization3, frames=100, interval=100, blit=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
figure4, axis4 = plt.subplots()
#axis4.hist(t, 10, label='Time to reach the goal', color='blue', lw=10)
axis4.legend()
axis4.set_title("Time elapsed before reaching the goal")
axis4.set_xlabel("Seconds")

def initialization4():
    axis4.hist(t, 10, label='Time to reach the goal', color='blue', lw=10)

def anim4(i):
    global t, axis4
    axis4.hist(t, 10, label='Time to reach the goal', color='blue', lw=10)
    
animation4 = animation.FuncAnimation(figure4, anim4, init_func=initialization4, frames=100, interval=100, blit=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
figure5, axis5 = plt.subplots()
figure6, axis6 = plt.subplots()

axis5.set_xlim((-10,10))
axis5.set_ylim((-10,10))
linee,=axis5.plot([],[], lw=1, color='green', label='Position')
axis5.legend()
axis5.set_title("Position")
#axis6.quiver(0,0,dir_x,dir_y,label="Orientation")
#axis6.legend()
#axis6.set_title("Orientation")

def initialization5():
    linee.set_data([],[])

def initialization6():
    axis6.quiver(0,0,dir_x,dir_y,label="Orientation")

def anim5(i):
    global x, y
    linee.set_data(x,y)
    return (linee)

def anim6(i):
    axis6.clear()
    axis6.quiver(0,0,dir_x,dir_y,label="Orientation")
    axis6.legend()
    axis6.set_title("Orientation")
    return(linee)

animation5 = animation.FuncAnimation(figure5, anim5, init_func=initialization5, frames=100, interval=100, blit=True)
animation6 = animation.FuncAnimation(figure6, anim6, init_func=initialization6, frames=1, interval=100, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …